In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

2024-09-14 15:02:36.503300: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 15:02:36.537930: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 15:02:36.537954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 15:02:36.538896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 15:02:36.544053: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 15:02:36.544619: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
#Mapping index back to words
word_index=imdb.get_word_index()
# use the key as index of the dictionary
index_from=3 # this is how the dataset is done... refer to keras documentation
reverse_word_index={i+index_from:word for word, i in word_index.items()}
reverse_word_index

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

In [4]:
model=load_model('simple_rnn_imdb.h5')
model.summary()

2024-09-14 15:03:44.832389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-14 15:03:44.832994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.get_weights()

[array([[ 0.0040599 ,  0.02702788,  0.02732901, ..., -0.04203946,
          0.01398647,  0.00112659],
        [-0.00057441,  0.05198982, -0.0650694 , ...,  0.03436185,
         -0.01981577, -0.01273304],
        [-0.0563491 ,  0.00626369,  0.02044766, ..., -0.0058137 ,
          0.01210763, -0.0591039 ],
        ...,
        [ 0.02932339,  0.00132445, -0.02483505, ...,  0.00189885,
         -0.01146972, -0.02870042],
        [ 0.03516814,  0.00437329, -0.00266756, ..., -0.04411874,
          0.03686451, -0.0327891 ],
        [-0.06090304,  0.03787132, -0.06527547, ..., -0.08038145,
         -0.0078007 , -0.06557314]], dtype=float32),
 array([[-0.00899814,  0.06789348,  0.02868566, ...,  0.10829473,
         -0.12860893, -0.13494731],
        [-0.09942795, -0.01938505, -0.11458392, ..., -0.01783798,
         -0.12733443, -0.1208017 ],
        [ 0.10170505,  0.00086937,  0.06531793, ...,  0.00307636,
         -0.15521635,  0.01184302],
        ...,
        [-0.13187097,  0.01643314, -0.0

In [7]:
(X_train, y_train),(X_test,y_test)=imdb.load_data(num_words=10000)

In [9]:
# Step 2: Helper functions
# functions to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i,'?') for i in encoded_review])

# function to preprocess user input
def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2) for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review


In [10]:
decode_review(X_train[0])

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [11]:
# prediction function
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    prediction=model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [12]:
example_review="This movie was fantastic! The acting was great and plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction score: {score}')

1/1 [==============================] - 0s 152ms/step
Review: This movie was fantastic! The acting was great and plot was thrilling.
Sentiment: Positive
Prediction score: 0.6437683701515198
